## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt
import matplotlib.colors as cl

import sys
sys.path.append('./lib')

from leaflet import *

In [10]:
data_dir = "../../Data/Weather/"
from pickle import load
# with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
with open(data_dir+'/SNWD_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()

['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [11]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.121373,0.158349,-0.017772,0.021817
1,-0.098271,0.199378,-0.025746,-0.057635
2,-0.113642,0.195043,-0.079399,0.201574
3,-0.090519,0.162509,-0.021666,-0.043858
4,-0.146439,0.203534,-0.055041,0.115309


In [12]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00213727', u'USC00212050', u'USC00214191', u'CA006020559', u'CA006034075']


,0,1,2,3
station,,,,
USC00213727,-0.121373,0.158349,-0.017772,0.021817
USC00212050,-0.098271,0.199378,-0.025746,-0.057635
USC00214191,-0.113642,0.195043,-0.079399,0.201574
CA006020559,-0.090519,0.162509,-0.021666,-0.043858
CA006034075,-0.146439,0.203534,-0.055041,0.115309


In [13]:
table=Eig.join(Stations,how='left')

In [14]:
table=table[['name','latitude','longitude','elevation',0,1,2,3,]]#4,5,6,7]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00213727,HIBBING PWR SUBSTN,47.4333,-92.9667,467.0,-0.121373,0.158349,-0.017772,0.021817
USC00212050,DEEP PORTAGE,46.8997,-94.3872,448.4,-0.098271,0.199378,-0.025746,-0.057635
USC00214191,KABETOGAMA,48.4089,-93.0483,365.8,-0.113642,0.195043,-0.079399,0.201574
CA006020559,BARWICK,48.6300,-93.9700,335.0,-0.090519,0.162509,-0.021666,-0.043858


In [17]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
print colors
# colors = [a.encode('ascii') for a in prop_cycle.by_key()['color']]

[u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']


## Map

In [24]:
# min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)
min_lat,max_lat,min_long,max_long = box = (46.3197, 49.833, -94.8297, -92.5253)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 7
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=cl.to_hex(color))
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=cl.to_hex(color))
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [9]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined